In [4]:
####
#Fiala fit
####

import numpy as np
import matplotlib.pyplot as plt

import fiala as f
import Pacejka52 as pa52

Fz = 4000
Fz0 = 3114
IA = 0

i=1000
Sx = np.linspace(-1,1,i)
Sy = np.linspace(-0.3,0.3,i)

params_Fx_init = [30000, 0.4, 0.6]
params_Fy_init = [11600, 0.2, 0.8]
Fx = f.fiala_long(Sy,Fz, *params_Fx_init)
Fy = f.fiala_lat(Sy,Fz, *params_Fy_init)

plt.figure(1)
plt.plot(Sx,Fx)

plt.figure(2)
plt.plot(Sy,Fy)

TypeError: fiala_long() missing 1 required positional argument: 'mu2'

In [2]:
#Fitting
import scipy.optimize as opt
from scipy.optimize import curve_fit
#Pacejka 52
IA = 0
Fz0 = 3114

Sx = np.linspace(0,0.15,1000)
Sy = np.linspace(0.01,0.115,1000)

params_Fx_init = [30000, 0.4, 0.6]
params_Fy_init = [11600, 0.2, 0.8]

##Fx_pa = pa.Pacejka52_long(Fz,Sx,IA,Fz0)[0]
Fy_pa = pa.Pacejka52_lat(Fz,Sy,IA,Fz0)[0]


#def Fx_br_func(Sx,mu,kt,lam):
#    return br.brush_long(Sx,Fz, mu, kt, lam)

def Fy_fi_func(Sy, C_lat,mu1,mu2):
    return f.fiala_lat(Sy,Fz,C_lat,mu1,mu2)

#Fx_br_params, _ = curve_fit(Fx_br_func, Sx, Fx_pa, params_Fx_init)
Fy_br_params, _ = curve_fit(Fy_fi_func, Sy, Fy_pa, params_Fy_init)

#Fx_br_fit = Fx_br_func(Sx,Fx_br_params[0],Fx_br_params[1],Fx_br_params[2])
Fy_br_fit = Fy_br_func(Sy,Fy_br_params[0],Fy_br_params[1], Fy_br_params[2])

#plt.figure(1)
#plt.plot(Sx,Fx_br_fit)
#plt.plot(Sx,Fx_pa)

plt.figure(2)
plt.plot(Sy,Fy_br_fit)
plt.plot(Sy,Fy_pa)


NameError: name 'pa' is not defined